In [2]:
import numpy as np
import pandas as pd
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
from keras.datasets import mnist

In [4]:
(train_image,train_label), (test_image, test_label) = mnist.load_data()

**数据预处理**

In [5]:
#注意当前CNN和MLP对数据的第一步预处理差别，因CNN要进行卷积运算，需保持数组样式，不能同MLP一样转成一维784个元素
train_image_4D = train_image.reshape(60000, 28, 28, 1).astype(float)   
test_image_4D = test_image.reshape(10000, 28, 28, 1).astype(float)

In [6]:
#标准化处理
train_image_4D_normalize = train_image_4D / 255
test_image_4D_normalize = test_image_4D / 255

In [7]:
#使用一位有效编码来处理标签
train_label_onehotencoding = np_utils.to_categorical(train_label)
test_label_onehotencoding = np_utils.to_categorical(test_label)

**建立CNN卷积神经网络**

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten   #分别对应卷积层，池化层和平坦层

In [9]:
model = Sequential()

添加卷积层1

In [10]:
#filter:滤镜个数  kernel_size:滤镜尺寸 padding=same表示不改变图片的大小 input_shape:输入图片的尺寸
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='same', input_shape=(28,28,1), activation='relu'))

添加池化层1

In [11]:
model.add(MaxPooling2D(pool_size = (2, 2)))   #将28*28的图片缩减一半变为14*14

添加卷积层2

In [12]:
#filter:滤镜个数  kernel_size:滤镜尺寸 padding=same表示不改变图片的大小 
model.add(Conv2D(filters=36, kernel_size=(5,5), padding='same', activation='relu'))

添加池化层2

In [13]:
model.add(MaxPooling2D(pool_size = (2, 2)))   #将28*28的图片缩减一半变为14*14

In [14]:
model.add(Dropout(0.25))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


添加平坦层

In [15]:
model.add(Flatten())

添加隐藏层

In [16]:
model.add(Dense(units=128, kernel_initializer='normal', activation='relu'))

In [17]:
model.add(Dropout(0.5))

建立隐藏层和输出层之间的关系

In [18]:
model.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

In [19]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               225920    
__________

In [20]:
#配置下训练模型  loss:损失函数  optimizer:优化器,通过优化让训练的结果能够更快的收敛 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
#train_image_normalize:训练数据 train_label_onehotencoding：训练标签 
#validation_split：取出一定比例的数据用于验证  epoch：训练次数 batch_size：每次训练取出多少数据用于训练
#verbose：2  表示显示训练过程
train_history = model.fit(train_image_4D_normalize, train_label_onehotencoding, validation_split=0.2, epochs=10, batch_size=200, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 48000 samples, validate on 12000 samples
Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
	 [[loss/mul/_115]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
print(train_history.history['acc'])
print('--------------------------------------------------------------------------------------------------------')
print(train_history.history['val_acc'])
print('--------------------------------------------------------------------------------------------------------')
print(train_history.history['loss'])
print('--------------------------------------------------------------------------------------------------------')
print(train_history.history['val_loss'])

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(history, train, validation):
    plt.plot(history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.xlabel('Epoch')
    plt.ylabel(train)
    plt.legend(['train', 'validation'], loc = 'upper left')
    plt.show()

In [ ]:
show_train_history(train_history, 'acc', 'val_acc')

In [ ]:
show_train_history(train_history, 'loss', 'val_loss')

利用10000项测试数据评估下模型的准确率

In [ ]:
scores = model.evaluate(test_image_4D_normalize, test_label_onehotencoding)
print(scores)

**导出训练模型**

In [ ]:
model.save('CNN_MNIST_model.h5')